In [1]:
import pandas
import re
from bs4 import BeautifulSoup as bs
import json
import requests
from requests.exceptions import RequestException
import time
import pandas as pd

df = pd.read_csv("t439.csv",index_col=0)

In [2]:
def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

def parse_one_page(html,ticker):
    time = []
    factor = []
    items = re.findall('<td style="text-align:center">(.*)</td>', html)
    n = int(len(items)/2)
    for i in range(11):
        time.append(items[i*2])
        factor.append(items[i*2+1].lstrip("$"))
    dic = {
        "time":time,
        ticker:factor
    }
    return dic

def restoreNumber(lst):
    """
    change the type of the data
    from str to float
    """
    new_lst = []
    for i in lst:
        if i:
            pattern=re.compile(',')
            numList=pattern.split(i)
            numStr=''.join(numList)
            num = float(numStr)
            new_lst.append(num)
        else:
            new_lst.append(i)
    return new_lst

In [3]:
html = get_one_page("https://www.macrotrends.net/stocks/charts/AAPL/apple/operating-income")
dff = pd.DataFrame(parse_one_page(html,"AAPL"))
dff

,time,AAPL
0,2020,"66,288"
1,2019,"63,930"
2,2018,"70,898"
3,2017,"61,344"
4,2016,"60,024"
5,2015,"71,230"
6,2014,"52,503"
7,2013,"48,999"
8,2012,"55,241"
9,2011,"33,790"


In [4]:
dictionary = {}
for i in range(len(df)):
    ticker = df["Symbol"][i]
    name = df["name"][i]
    url = "https://www.macrotrends.net/stocks/charts/{}/{}/operating-income".format(ticker,name)
    #print(url)
    try:
        html = get_one_page(url)
        
        d = parse_one_page(html,ticker)
        dictionary.update(d)
    except:
        print(ticker)
roe = pd.DataFrame(dictionary)    
roe.to_csv("operating_income.csv") 
roe

,time,MMM,ABT,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WU,WY,WHR,WMB,WYNN,XEL,XLNX,YUM,ZBRA,ZBH
0,2020,"7,161","5,357",249,"6,514","2,734","4,237","1,369",750,690,...,967,"1,710","1,623","2,202","-1,232","2,116",792,"1,503",651,-88
1,2019,"6,174","4,532",225,"6,305","1,607","3,268",631,677,"1,093",...,934,651,"1,571","1,921",878,"2,104",957,"1,930",692,"1,138"
2,2018,"7,207","3,650",157,"5,899","1,988","2,840",451,604,"2,165",...,"1,122","1,394",279,768,736,"1,965",686,"2,296",610,34
3,2017,"7,692","1,564",90,"5,191","1,309","2,168",127,570,771,...,476,"1,131","1,136",927,"1,056","2,223",706,"2,761",322,799
4,2016,"7,027","3,026",65,"4,810","1,412","1,494",-373,788,189,...,487,822,"1,368",689,522,"2,240",670,"1,682",80,821
5,2015,"6,946","2,867",29,"4,436","1,319",903,-481,826,989,...,"1,109",644,"1,242",226,659,"2,000",755,"1,434",37,467
6,2014,"7,135","2,599",8,"4,301","1,183",413,-155,852,"1,571",...,"1,141",987,"1,188","1,569","1,266","1,948",749,"1,517",89,"1,037"
7,2013,"6,666","2,133",17,"4,339","1,372",423,103,660,"1,177",...,"1,107",634,"1,249","1,375","1,290","1,848",581,"1,798",160,"1,069"
8,2012,"6,483","1,360",2,"3,872","1,451","1,180","-1,056",657,280,...,"1,330",618,869,"1,612","1,029","1,823",628,"2,294",164,"1,047"
9,2011,"6,178","1,629",-11,"3,470","1,328","1,099",368,665,"2,342",...,"1,385",594,792,"1,867","1,008","1,782",795,"1,815",182,"1,024"


In [45]:
df = pd.read_csv("operating_income_macro.csv",index_col=1,parse_dates=True)
df.drop(columns="Unnamed: 0",inplace=True)
df = df.sort_index()
df.to_csv("ope_in_final.csv")